# Process [journalmetrics](https://journalmetrics.scopus.com/) data into tidy TSVs

In [1]:
import os
import gzip

import pandas

## Prepare a dataframe of metrics and values

In [2]:
path = os.path.join('download', 'CiteScore_Metrics_2011-2105_Download_19_Jan2017.xlsx')
xlsx = pandas.ExcelFile(path)
sheets = [sheet for sheet in xlsx.sheet_names if ' All' in sheet]
sheets

['2015 All', '2014 All', '2013 All', '2012 All', '2011 All']

In [3]:
renamer = {
    'Scopus SourceID': 'scopus_id',
}

metrics = ['CiteScore', 'SNIP', 'SJR']

dfs = list()
for sheet in sheets:
    year, _ = sheet.split()
    year = int(year)

    df = (
        xlsx.parse('2015 All', skiprows=1)
        .rename(columns=renamer)
        .drop_duplicates(['scopus_id'] + metrics)
        .assign(year=year)
    )

    df = pandas.melt(df, id_vars=['scopus_id', 'year'], value_vars=metrics, var_name='metric').dropna()
    dfs.append(df)

metric_df = pandas.concat(dfs).sort_values(['scopus_id', 'year', 'metric'])

In [4]:
len(metric_df)

329110

In [5]:
metric_df.head(2)

,scopus_id,year,metric,value
3301,12001,2011,CiteScore,2.340
47813,12001,2011,SJR,0.998


In [6]:
metric_df.metric.value_counts()

CiteScore    111280
SJR          110220
SNIP         107610
Name: metric, dtype: int64

In [7]:
metric_df.year.value_counts()

2015    65822
2014    65822
2013    65822
2012    65822
2011    65822
Name: year, dtype: int64

In [8]:
with gzip.open('data/metrics.tsv.gz', 'wt') as write_file:
    metric_df.to_csv(write_file, sep='\t', index=False, float_format='%.4g')